In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
df = pd.read_csv('male_players.csv')

print(f"DATASET OVERVIEW")

print(f"Total Players: {len(df):,}")
print(f"Total Features: {df.shape[1]}")
print(f"\nFirst few rows:")
print(df.head())

# Data quality check
print(f"DATA QUALITY CHECK")
print(f"\nMissing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0])

print(f"Data Types:")
print(df.dtypes.value_counts())

# Basic statistics
print(f"KEY STATISTICS")

print(df[['OVR', 'Age', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']].describe())

DATASET OVERVIEW
Total Players: 16,161
Total Features: 58

First few rows:
   Unnamed: 0.1  Unnamed: 0  Rank             Name  OVR  PAC  SHO  PAS  DRI  \
0             0           0     1    Kylian Mbappé   91   97   90   80   92   
1             1           1     2            Rodri   91   66   80   86   84   
2             2           2     4   Erling Haaland   91   88   92   70   81   
3             3           3     5  Jude Bellingham   90   80   87   83   88   
4             4           4     7         Vini Jr.   90   95   84   81   91   

   DEF  PHY  Acceleration  Sprint Speed  Positioning  Finishing  Shot Power  \
0   36   78            97            97           93         94          90   
1   87   85            65            66           76         74          92   
2   45   88            80            94           96         96          94   
3   78   83            81            80           91         90          85   
4   29   69            95            95           87   

In [60]:
print(f"DATA CLEANING")


# Create a clean copy
df_clean = df.copy()

# Remove goalkeepers for outfield analysis 
df_outfield = df_clean[df_clean['Position'] != 'GK'].copy()
df_gk = df_clean[df_clean['Position'] == 'GK'].copy()

print(f"Outfield Players: {len(df_outfield):,}")
print(f"Goalkeepers: {len(df_gk):,}")

# Handle missing values (if any)
numeric_cols = df_outfield.select_dtypes(include=[np.number]).columns
df_outfield[numeric_cols] = df_outfield[numeric_cols].fillna(df_outfield[numeric_cols].median())

# Create age categories
df_outfield['Age_Category'] = pd.cut(df_outfield['Age'], 
                                      bins=[0, 21, 25, 29, 32, 50],
                                      labels=['Young Prospect (≤21)', 'Developing (22-25)', 
                                             'Prime (26-29)', 'Experienced (30-32)', 'Veteran (33+)'])

# Create rating tiers
df_outfield['Rating_Tier'] = pd.cut(df_outfield['OVR'],
                                     bins=[0, 70, 75, 80, 85, 100],
                                     labels=['Below Average (<70)', 'Average (70-74)', 
                                            'Good (75-79)', 'Top (80-84)', 'Elite (85+)'])

print("Data cleaning completed!")

DATA CLEANING
Outfield Players: 14,345
Goalkeepers: 1,816
Data cleaning completed!


In [38]:
# Calculate performance index for strikers
def calculate_striker_index(row):    
    return (
        row['Finishing'] * 0.30 +
        row['Positioning'] * 0.25 +
        row['Shot Power'] * 0.15 +
        row['PAC'] * 0.15 +
        row['Composure'] * 0.10 +
        row['Dribbling'] * 0.05
    )

# Calculate performance index for wingers
def calculate_winger_index(row):
    return (
        row['PAC'] * 0.30 +
        row['Dribbling'] * 0.25 +
        row['Crossing'] * 0.20 +
        row['Agility'] * 0.15 +
        row['Finishing'] * 0.10
    )

# Calculate performance index for midfielders
def calculate_midfielder_index(row):
    return (
        row['PAS'] * 0.30 +
        row['Vision'] * 0.25 +
        row['Ball Control'] * 0.20 +
        row['Stamina'] * 0.15 +
        row['Long Passing'] * 0.10
    )
    
# Calculate performance index for defenders
def calculate_defender_index(row):
    return (
        row['DEF'] * 0.30 +
        row['Standing Tackle'] * 0.25 +
        row['Heading Accuracy'] * 0.15 +
        row['Strength'] * 0.15 +
        row['Interceptions'] * 0.10 +
        row['PAC'] * 0.05
    )

In [39]:
# Position groupings
strikers = ['ST', 'CF', 'LF', 'RF']
wingers = ['LW', 'RW', 'LM', 'RM']
midfielders = ['CM', 'CDM', 'CAM']
defenders = ['CB', 'LB', 'RB', 'LWB', 'RWB']

In [59]:
# Calculate indices
df_outfield['Position_Performance_Index'] = 0.0

for idx, row in df_outfield.iterrows():
    if row['Position'] in strikers:
        df_outfield.at[idx, 'Position_Performance_Index'] = calculate_striker_index(row)
        df_outfield.at[idx, 'Position_Group'] = 'Striker'
    elif row['Position'] in wingers:
        df_outfield.at[idx, 'Position_Performance_Index'] = calculate_winger_index(row)
        df_outfield.at[idx, 'Position_Group'] = 'Winger'
    elif row['Position'] in midfielders:
        df_outfield.at[idx, 'Position_Performance_Index'] = calculate_midfielder_index(row)
        df_outfield.at[idx, 'Position_Group'] = 'Midfielder'
    elif row['Position'] in defenders:
        df_outfield.at[idx, 'Position_Performance_Index'] = calculate_defender_index(row)
        df_outfield.at[idx, 'Position_Group'] = 'Defender'
    else:
        df_outfield.at[idx, 'Position_Performance_Index'] = row['OVR']
        df_outfield.at[idx, 'Position_Group'] = 'Other'

print("Position-specific indices calculated!")
print(f"\nSample Performance Indices:")
print(df_outfield[['Name', 'Position', 'OVR', 'Position_Performance_Index']].head(10))


Position-specific indices calculated!

Sample Performance Indices:
                Name Position  OVR  Position_Performance_Index
0      Kylian Mbappé       ST   91                       92.95
1              Rodri      CDM   91                       87.55
2     Erling Haaland       ST   91                       92.75
3    Jude Bellingham      CAM   90                       88.05
4           Vini Jr.       LW   90                       90.95
5    Kevin De Bruyne       CM   90                       92.85
6         Harry Kane       ST   90                       89.15
7    Martin Ødegaard       CM   89                       89.75
11  Lautaro Martínez       ST   89                       89.30
12   Virgil van Dijk       CB   89                       89.15


In [ ]:
print(f"CALCULATING VALUE SCORES")

# Value Score: How much better a player performs vs their rating
df_outfield['Value_Score'] = (
    (df_outfield['Position_Performance_Index'] - df_outfield['OVR']) / df_outfield['OVR']
) * 100

# League tier for bonus calculation
top_5_leagues = ['Premier League', 'La Liga', 'Serie A', 'Bundesliga', 'Ligue 1']
df_outfield['League_Bonus'] = df_outfield['League'].apply(
    lambda x: 0 if x in top_5_leagues else 10
)

# Hidden Gem Index 
df_outfield['Hidden_Gem_Index'] = (
    df_outfield['Value_Score'] * 0.40 +  # 40% weight on value
    (30 - df_outfield['Age']) * 0.30 +   # 30% weight on youth
    df_outfield['League_Bonus'] * 0.15 + # 15% weight on league
    (79 - df_outfield['OVR'].clip(upper=79)) * 0.15  # 15% weight on being underrated
)

# Performance vs Rating Classification
df_outfield['Performance_vs_Rating'] = pd.cut(
    df_outfield['Value_Score'],
    bins=[-100, -5, 0, 5, 10, 100],
    labels=['Significantly Overvalued', 'Overvalued', 'Fairly Valued', 'Undervalued', 'Hidden Gem']
)

# Quadrant Classification (Age vs Value)
def classify_quadrant(row):
    if row['Value_Score'] >= 5 and row['Age'] <= 25:
        return ' Prime Target'
    elif row['Value_Score'] >= 5 and row['Age'] > 25:
        return 'Short-term Value'
    elif row['Value_Score'] < 5 and row['Age'] <= 25:
        return 'Development Player'
    else:
        return 'Overvalued'

df_outfield['Quadrant'] = df_outfield.apply(classify_quadrant, axis=1)

print("Value scores calculated!")
print(f"\nValue Score Distribution:")
print(df_outfield['Value_Score'].describe())
print(f"\nPerformance vs Rating Distribution:")
print(df_outfield['Performance_vs_Rating'].value_counts())

CALCULATING VALUE SCORES
Value scores calculated!

Value Score Distribution:
count   14345.00
mean        0.27
std         5.11
min       -21.85
25%        -2.32
50%         0.83
75%         2.95
max        21.30
Name: Value_Score, dtype: float64

Performance vs Rating Distribution:
Performance_vs_Rating
Fairly Valued               6624
Overvalued                  3641
Significantly Overvalued    2220
Undervalued                 1400
Hidden Gem                   460
Name: count, dtype: int64


In [57]:
print(f"IDENTIFYING TOP HIDDEN GEMS")


# Filter criteria for hidden gems
hidden_gems = df_outfield[
    (df_outfield['Value_Score'] > 5) &  # Must have positive value
    (df_outfield['OVR'] <= 82) &  # Not already highly rated
    (df_outfield['Age'] <= 28)  # Reasonable age
].copy()

# Sort by Hidden Gem Index
hidden_gems_sorted = hidden_gems.sort_values('Hidden_Gem_Index', ascending=False)

# Top 50 Hidden Gems
top_50_gems = hidden_gems_sorted.head(50)

print(f"Total Hidden Gems Found: {len(hidden_gems):,}")
print(f"TOP 10 HIDDEN GEMS:")


display_cols = ['Name', 'Position', 'Age', 'OVR', 'Position_Performance_Index', 
                'Value_Score', 'Hidden_Gem_Index', 'Team', 'League', 'Nation']

print(top_50_gems[display_cols].head(10).to_string(index=False))

IDENTIFYING TOP HIDDEN GEMS
Total Hidden Gems Found: 1,577
TOP 10 HIDDEN GEMS:
             Name Position  Age  OVR  Position_Performance_Index  Value_Score  Hidden_Gem_Index             Team             League         Nation
        Li Deming       RW   20   50                       60.65        21.30             17.37 Shanghai Port FC                CSL       China PR
       Thoi Singh       LW   20   49                       58.40        19.18             16.67 NorthEast United                ISL          India
    Eom Seung Min       RW   21   52                       62.70        20.58             16.48  Jeonbuk Hyundai         K League 1 Korea Republic
      Serge Ngoma       RM   19   58                       70.30        21.21             16.43        Red Bulls                MLS  United States
    Kyrell Wilson       RM   19   53                       62.45        17.83             15.83     Swansea City   EFL Championship        England
    Qeyser Ezimet       RW   18   48   

In [56]:
print(f"POSITION-SPECIFIC HIDDEN GEMS")

position_groups = ['Striker', 'Winger', 'Midfielder', 'Defender']

position_gems = {}
for pos_group in position_groups:
    pos_data = hidden_gems[hidden_gems['Position_Group'] == pos_group].sort_values(
        'Hidden_Gem_Index', ascending=False
    ).head(10)
    position_gems[pos_group] = pos_data
    
    print(f"TOP 10 {pos_group.upper()} HIDDEN GEMS:")
    print(pos_data[['Name', 'Position', 'Age', 'OVR', 'Value_Score', 'Team']].to_string(index=False))

POSITION-SPECIFIC HIDDEN GEMS
TOP 10 STRIKER HIDDEN GEMS:
                    Name Position  Age  OVR  Value_Score            Team
Felipe Rodríguez-Gentile       ST   17   57         8.86         Preston
        Fardin Ali Molla       ST   22   49         9.49  Mohun Bagan SG
              Luke Supyk       ST   18   54         7.87   Well. Phoenix
             Gbemi Arubi       ST   20   52         8.46       Waterford
            Makise Evans       ST   18   51         6.37       Stevenage
          Mohammed Aimen       ST   21   47         6.81 Kerala Blasters
         Bridel Bosakani       ST   20   50         7.00 Drogheda United
              Eoin Kenny       ST   18   52         6.15         Dundalk
            Rory MacLeod       ST   18   50         5.30   Dundee United
                Tom King       ST   17   52         5.19 Cheltenham Town
TOP 10 WINGER HIDDEN GEMS:
             Name Position  Age  OVR  Value_Score             Team
        Li Deming       RW   20   50        2

In [55]:
print(f"LEAGUE & NATION ANALYSIS")

# Hidden gems by league
league_analysis = hidden_gems.groupby('League').agg({
    'Name': 'count',
    'Hidden_Gem_Index': 'mean',
    'Value_Score': 'mean',
    'Age': 'mean'
}).round(2)

league_analysis.columns = ['Hidden_Gems_Count', 'Avg_HG_Index', 'Avg_Value_Score', 'Avg_Age']
league_analysis = league_analysis.sort_values('Hidden_Gems_Count', ascending=False).head(15)

print("TOP LEAGUES FOR HIDDEN GEMS:")
print(league_analysis)

# Hidden gems by nation
nation_analysis = hidden_gems.groupby('Nation').agg({
    'Name': 'count',
    'Hidden_Gem_Index': 'mean',
    'Value_Score': 'mean'
}).round(2)

nation_analysis.columns = ['Hidden_Gems_Count', 'Avg_HG_Index', 'Avg_Value_Score']
nation_analysis = nation_analysis.sort_values('Hidden_Gems_Count', ascending=False).head(15)

print("TOP NATIONS PRODUCING HIDDEN GEMS:")
print(nation_analysis)

LEAGUE & NATION ANALYSIS
TOP LEAGUES FOR HIDDEN GEMS:
                    Hidden_Gems_Count  Avg_HG_Index  Avg_Value_Score  Avg_Age
League                                                                       
MLS                               100          9.93             8.57    22.31
K League 1                         79         10.50             8.69    22.52
Sudamericana                       74          9.38             8.26    22.82
EFL Championship                   71          9.41             8.61    22.00
CSL                                58          9.65             8.89    24.81
EFL League Two                     58         10.36             9.74    23.17
PKO BP Ekstraklasa                 55          9.47             8.55    23.16
Libertadores                       54          8.62             7.83    23.44
Eredivisie                         54          9.21             8.41    22.20
3. Liga                            52          9.67             8.36    22.71
EFL League

In [54]:
print(f"EXPORTING PROCESSED DATA")

# Create processed directory if it doesn't exist
import os
os.makedirs('data/processed', exist_ok=True)

# Export main dataset with all metrics
df_outfield.to_csv('data/processed/players_with_metrics.csv', index=False)
print("Exported: players_with_metrics.csv")

# Export top 50 hidden gems
top_50_gems.to_csv('data/processed/hidden_gems_top50.csv', index=False)
print("Exported: hidden_gems_top50.csv")

# Export position-specific analysis
position_summary = df_outfield.groupby('Position_Group').agg({
    'OVR': 'mean',
    'Position_Performance_Index': 'mean',
    'Value_Score': 'mean',
    'Age': 'mean',
    'Name': 'count'
}).round(2)
position_summary.to_csv('data/processed/position_analysis.csv')
print("Exported: position_analysis.csv")

# Export league insights
league_analysis.to_csv('data/processed/league_insights.csv')
print("Exported: league_insights.csv")

# Export nation insights
nation_analysis.to_csv('data/processed/nation_insights.csv')
print("Exported: nation_insights.csv")

# Export correlation matrix
correlation_matrix.to_csv('data/processed/correlation_matrix.csv')
print("Exported: correlation_matrix.csv")


print("ANALYSIS COMPLETE!")



EXPORTING PROCESSED DATA
Exported: players_with_metrics.csv
Exported: hidden_gems_top50.csv
Exported: position_analysis.csv
Exported: league_insights.csv
Exported: nation_insights.csv
Exported: correlation_matrix.csv
ANALYSIS COMPLETE!


In [53]:
print("KEY FINDINGS SUMMARY:")
print(f"• Total Players Analyzed: {len(df_outfield):,}")
print(f"• Hidden Gems Identified: {len(hidden_gems):,}")
print(f"• Average Value Score: {df_outfield['Value_Score'].mean():.2f}%")
print(f"• Best League for Hidden Gems: {league_analysis.index[0]}")
print(f"• Most Undervalued Position: {position_value['mean'].idxmax()}")
print(f"All processed files saved to: data/processed/")
print(f"Ready to import into Power BI!")

KEY FINDINGS SUMMARY:
• Total Players Analyzed: 14,345
• Hidden Gems Identified: 1,577
• Average Value Score: 0.27%
• Best League for Hidden Gems: MLS
• Most Undervalued Position: Winger
All processed files saved to: data/processed/
Ready to import into Power BI!
